In [23]:
from sedona.spark import *
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)
from pyspark.sql.functions import col, expr

25/04/24 12:47:32 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/04/24 12:47:32 WARN SimpleFunctionRegistry: The function rs_union_aggr replaced a previously registered function.
25/04/24 12:47:32 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/04/24 12:47:32 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.geometryObjects.Geography, which is already registered.
25/04/24 12:47:32 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/04/24 12:47:32 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously registered function.
25/04/24 12:47:32 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
25/04/24 12:47:32 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [36]:
parquet_path = "s3a://wbts-wbc-ve0p9to9ga/453obow93s/data/catalogs/wherobots/beatka/gdelt/data/00000-4719-e7a6239c-d66d-4ca4-a592-db23865b4a5e-0-00001.parquet"

In [37]:
df = sedona.read.parquet(parquet_path)

In [38]:
df.printSchema()

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: stri

In [59]:
df_selected = df.select("geometry")

In [61]:
df_selected.show(3)

+--------------------+
|            geometry|
+--------------------+
|[00 20 00 00 01 0...|
|[00 20 00 00 01 0...|
|[00 20 00 00 01 0...|
+--------------------+
only showing top 3 rows



In [62]:
# Funkcja pomocnicza do dekodowania geometrycznych danych binarnych na WKT (Well-Known Text)
def decode_geometry(wkb_col):
    return F.expr(f"ST_AsText(ST_GeomFromWKB({wkb_col}))")

In [65]:
# Zdekodowanie kolumny geometry
df_decoded = df_selected.withColumn("geometry_wkt", decode_geometry("geometry"))


In [68]:
df_decoded.printSchema()

root
 |-- geometry: binary (nullable = true)
 |-- geometry_wkt: string (nullable = true)



In [67]:
# Utwórz nową kolumnę "geometry" z kolumny WKT
df_with_geometry = df_decoded.withColumn("geometry1", F.expr("ST_GeomFromWKT(geometry_wkt)"))

In [70]:
df_with_geometry.printSchema()

root
 |-- geometry: binary (nullable = true)
 |-- geometry_wkt: string (nullable = true)
 |-- geometry1: geometry (nullable = true)



In [71]:
df_with_geometry.show()

+--------------------+--------------------+--------------------+
|            geometry|        geometry_wkt|           geometry1|
+--------------------+--------------------+--------------------+
|[00 20 00 00 01 0...|POINT (-86.8073 3...|POINT (-86.8073 3...|
|[00 20 00 00 01 0...|POINT (-73.9662 4...|POINT (-73.9662 4...|
|[00 20 00 00 01 0...|       POINT (38 35)|       POINT (38 35)|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|
|[00 20 00 00 01 0...|POINT (-157.531 2...|POINT (-157.531 2...|
|[00 20 00 00 01 0...|      POINT (138 36)|      POINT (138 36)|
|[00 20 00 00 01 0...|     POINT (133 -27)|     POINT (133 -27)|
|[00 20 00 00 01 0...| POINT (146.833 -18)| POINT (146.833 -18)|
|[00 20 00 00 01 0...|POINT (51.4128 31...|POINT (51.4128 31...|
|[00 20 00 00 01 0...|POI

In [72]:
df_decoded1 = df_with_geometry.withColumn(
    "geometry_tekst",
    F.decode(col("geometry"), "utf-8")  # Dekodowanie kolumny `geometry` jako tekst
)

In [73]:
df_decoded1.show()

+--------------------+--------------------+--------------------+--------------------+
|            geometry|        geometry_wkt|           geometry1|      geometry_tekst|
+--------------------+--------------------+--------------------+--------------------+
|[00 20 00 00 01 0...|POINT (-86.8073 3...|POINT (-86.8073 3...|      ��U��͞��@...|
|[00 20 00 00 01 0...|POINT (-73.9662 4...|POINT (-73.9662 4...|      ��R}�8�YK...|
|[00 20 00 00 01 0...|       POINT (38 35)|       POINT (38 35)|      �@C      ...|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|      ��RV��f�B...|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|      ��RV��f�B...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|      ��R3+\fI...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|      ��R3+\fI...|
|[00 20 00 00 01 0...|POINT (-157.531 2...|POINT (-157.531 2...|      ��c���E�@...|
|[00 20 00 00 01 0...|      POINT (138 36)|      POINT

In [75]:
df_renamed = (
    df_decoded1.withColumnRenamed("geometry", "binary")
      .withColumnRenamed("geometry1", "geometry")
)

In [76]:
df_renamed.show()

+--------------------+--------------------+--------------------+--------------------+
|              binary|        geometry_wkt|            geometry|      geometry_tekst|
+--------------------+--------------------+--------------------+--------------------+
|[00 20 00 00 01 0...|POINT (-86.8073 3...|POINT (-86.8073 3...|      ��U��͞��@...|
|[00 20 00 00 01 0...|POINT (-73.9662 4...|POINT (-73.9662 4...|      ��R}�8�YK...|
|[00 20 00 00 01 0...|       POINT (38 35)|       POINT (38 35)|      �@C      ...|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|      ��RV��f�B...|
|[00 20 00 00 01 0...|POINT (-73.3579 4...|POINT (-73.3579 4...|      ��RV��f�B...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|      ��R3+\fI...|
|[00 20 00 00 01 0...|POINT (-72.7995 4...|POINT (-72.7995 4...|      ��R3+\fI...|
|[00 20 00 00 01 0...|POINT (-157.531 2...|POINT (-157.531 2...|      ��c���E�@...|
|[00 20 00 00 01 0...|      POINT (138 36)|      POINT

In [79]:
df_geometry_only = df_renamed.select("geometry_wkt", "geometry", "geometry_tekst")

In [80]:
SedonaKepler.create_map(df_geometry_only, "Mapa Globalnych Zdarzeń")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Mapa Globalnych Zdarzeń': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…